# Variáveis categóricas

- Objetivo de organizar os dados a serrem analisados pelo modelo.
- Torna possível para o modelo avaliar os dados.
- Explicação com vídeo do youtube: `https://www.youtube.com/watch?v=TwVBewyylfk`

## Três abordagens:

### 1) Remover a coluna com variáveis categóricas

Útil somente se essa coluna não conter informações úteis.

### 2) Codificação ordinal (Ordinal Encoding)
- Assume um valor inteiro para cada tipo de variável categórica.
- Ordenamos os inteiros de acordo com a intensidade da variável.
- _Exemplo:_ <br> "Nunca" (0) < "Raramente" (1) < "Sempre" (2)

### 3) Codificação One-Hot (One-Hot Encoding)
- Cria uma coluna diferente para cada tipo único de variável categória
- Usado quando não é possível ordenar de acordo com uma intensidade ou relação direta. 
- _Exemplo:_ <br> Temos um dataframe com a seguinte coluna: "Cores" e com as variáveis: "Verde", "Amarelo" e "Vermelho". <br> Criamos 3 novas colunas no dataframe chamadas "Verde", "Amarelo" e "Vermelho". <br> Os dados nelas são do tipo inteiro. <br> Caso na colunas "Cores" esteja aparecendo "vermelho", colocamos o número 1 na nova coluna "Vermelho" e 0 nas colunas "Verde" e "Amarelo". <br> Após fazer isso para todas as linhas, excluimos a coluna "Cores" original do dataframe.

## Conceito: Cardinalidade

Significa o número de valores únicos em uma coluna. 

Selecionar colunas categóricas com baixa cardinalidade é conveniente, mas arbitrário.

In [ ]:


import pandas as pd
from sklearn.model_selection import train_test_split

# lendo os dados
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

# separando features e alvo
y = data.Price
X = data.drop(['Price'], axis=1)

# dividindo os dados para validação e treinamento
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# removendo colunas com valores NaN (abordagem mais simples)
colunas_faltantes = []

for col in X_train_full.columns:
    if X_train_full[col].isnull().any():
        colunas_faltantes.append(col)
X_train_full.drop(colunas_faltantes, axis=1, inplace=True)
X_valid_full.drop(colunas_faltantes, axis=1, inplace=True)

# vamos selecionar apenas colunas com baixa cardinalidade e também as numéricas para um melhor modelo

# baixa cardinalidade
low_cardinality_cols = []
for cname in X_train_full.columns:
    if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object":
        low_cardinality_cols.append(cname)

# colunas numéricas
numerical_cols = []
for cname in X_train_full.columns:
    if X_train_full[cname].dtype in ['int64', 'float64']:
        numerical_cols.append(cname)

# colunas selecionadas
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

Analisando quais variáveis são categóricas (são do tipo object):

In [ ]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Para testar cada uma das abordagens, usamos a função abaixo para comprar o erro médio absoluto:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

## Primeira abordagem: Remover variáveis categóricas

In [ ]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

## Segunda abordagem: Ordinal Encoding

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# cópia dos dados originais
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# aplicando a ordinal encoding em cada coluna com variáveis categóricas
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

## Terceira Abordagem: One-Hot Encoding
- `object_cols` na cédula abaixo, contém o nome das colunas com variáveis categórias.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# aplicando a one-hot encoding em cada coluna categórica
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False) # Nota: handle_unknown='ignore' assegura que não haverá erros e sparse=False garante o retorno por um array numpy
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# one-hot encoding remove índices das colunas, colocamos eles de volta
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# removendo as colunas do dataframe original que contia as variáveis categóricas
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Cria as novas colunas a partir da one-hot com inteiros
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Transforma os nomes das colunas em tipo string
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))